# Legal Assertion Status Model 

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/12.Assertion_Status.ipynb)

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

In [ ]:
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## Legal Assertion Status Model 

The model has been implemented within Spark NLP as an annotator called **AssertionDLModel**. It is an Assertion Status Model aimed to detect **temporality (PRESENT, PAST, FUTURE)** or **certainty (POSSIBLE)** in your legal texts.

## Prediction Pipeline

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings_ner = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base", "en") \
    .setInputCols("sentence", "token") \
    .setOutputCol("embeddings_ner")\

ner_model = legal.NerModel.pretrained('legner_contract_doc_parties', 'en', 'legal/models')\
    .setInputCols(["sentence", "token", "embeddings_ner"])\
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(["DOC", "EFFDATE", "PARTY"])

embeddings_ass = BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings_ass")

assertion = legal.AssertionDLModel.pretrained("legassertion_time", "en", "legal/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings_ass"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
            document_assembler, 
            sentence_detector,
            tokenizer,
            embeddings_ner,
            ner_model,
            ner_converter,
            embeddings_ass,
            assertion
            ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

### Getting Result 

In [ ]:
sample_text = "This is an Intellectual Property Agreement between Amazon Inc. and Atlantic Inc."


In [ ]:
data = spark.createDataFrame([[sample_text]]).toDF("text")

data.show(truncate = 80)

+--------------------------------------------------------------------------------+
|                                                                            text|
+--------------------------------------------------------------------------------+
|This is an Intellectual Property Agreement between Amazon Inc. and Atlantic Inc.|
+--------------------------------------------------------------------------------+



In [ ]:
result = model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,  
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end, 
                                     result.ner_chunk.metadata, 
                                     result.assertion.result)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['4']").alias("assertion")).show(truncate=False)

+-------------------------------+-----+---+---------+---------+
|chunk                          |begin|end|ner_label|assertion|
+-------------------------------+-----+---+---------+---------+
|Intellectual Property Agreement|11   |41 |DOC      |PRESENT  |
|Amazon Inc                     |51   |60 |PARTY    |PRESENT  |
|Atlantic Inc                   |67   |78 |PARTY    |PRESENT  |
+-------------------------------+-----+---+---------+---------+



In [ ]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(result.collect()[0], 'ner_chunk', 'assertion')

### Getting Result with LightPipeline

In [ ]:
sample_text = ["""This TRADEMARK LICENSE AGREEMENT (this "Agreement") is made and effective as of 31 Aug, 2020 ("Effective Date"), by and between Palmer Square Capital Management LLC, a Delaware limited liability company (the "Licensor"), and Palmer Square Capital BDC Inc., a corporation organized under the laws of the State of Maryland (the "Licensee")""",
               """The Intellectual Property Agreement would potentially be in short signed by the two Parties""",
               """The Sponsorship Agreement ("Agreement") will be signed on October 10, 2015""",
               """This is an Intellectual Property Agreement between Amazon.com Inc and Atlantic Inc.""",
               """This Sponsorship Agreement ("Agreement") was entered into as of December 18, 1998, by and between Ford Motor Media, a division of J. Walter Thompson with offices at 300 Renaissance Center, Detroit, Michigan 48243 and iVillage Inc., with offices at 170 Fifth Avenue, New York, New York 10010."""]

In [ ]:
chunks=[]
entities=[]
status=[]
begin = []
end = []

for i in range(len(sample_text)):

    light_result = light_model.fullAnnotate(sample_text)[i]

    for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
        begin.append(n.begin)
        end.append(n.end)
        chunks.append(n.result)
        entities.append(n.metadata['entity']) 
        status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'begin':begin, 'end':end, 'ner_label':entities, 'assertion':status})

### Visualization of Assertion Status

In [ ]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

for i in range(len(sample_text)):
    
    light_result = light_model.fullAnnotate(sample_text)[i]
    
    vis = AssertionVisualizer()

    vis.display(light_result, 'ner_chunk', 'assertion')
    
    print("\n\n")